#Acquire Sentinel-2 Ground Truth Data from Cropland.org

This notebook is used for gathering ground truth data from cropland.org from the Sentinel-2 satellites. Specifically, we are looking to acquire the surface reflectance data (atmosphere corrected - level 2a) as we did with our California data. We will be looking at the ground truth data downloaded for years 2010 onwards and only pull out satellite images for the records we are interested in. We have the lat, lon, month and year of the data available.

In [1]:
import time
from math import sin, cos, sqrt, atan2, radians
import pandas as pd
import ee
from shapely.geometry import box
import folium
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import find_peaks

## Authenticate on EE

To be able to download anything we first need to authenticate ourselves on the EE.

In [24]:
def authenticate():
    # Trigger the authentication flow.
    ee.Authenticate()

authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=SjHfetoq0chv7wIyNMj1Fdd_KvJK3d7gmBoio-49BP8&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g4eKnc-ihqIQBTEkCurbsLGTlzMwuAcT0a0rksaiI28lgPyHpY8KBc

Successfully saved authorization token.


## Load Google Drive

Load our drive to be able to load the csv data

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [23]:
import sys
import os
sys.path.append('/content/gdrive/My Drive/capstone_code')
os.chdir('/content/gdrive/My Drive/capstone_code')
os.listdir()

['ground_truth_cropland.csv',
 'gather_california_data.ipynb',
 'CroplandDataSet',
 'GroundTruthEDA.ipynb',
 'FilterDuplicateLocations.py',
 'ground_truth_data_10.csv',
 'ground_truth_data_100.csv',
 'Irrigated',
 'Sentinel2_ground_truth_data.ipynb',
 'Rainfed',
 'LANDSAT8_ground_truth_data.ipynb',
 'USIrrigated',
 'USRainfed']

## Load the ground truth meta data
Load the csv file and filter based on our records of interest. Our label is `water` a binary value: Irrigated land (code 1) vs rainfed (code 2)

In [42]:
filename = 'ground_truth_data_10.csv'
data = pd.read_csv(filename)
data.head()

,Unnamed: 0,id,year,month,lat,lon,country,land_use_type,crop_primary,crop_secondary,water,intensity,source_type,source_class,source_description,use_validation
0,0,172978,2020,0,50.863663,4.359379,Belgium,1,0,1,1,2,ground,NaN,mobile_application,False
1,1,172961,2019,8,35.606363,-112.097508,United States of America,1,20,0,2,1,ground,NaN,mobile_application,False
2,4,172953,2019,8,35.177561,-111.680403,United States of America,1,0,0,1,0,ground,NaN,mobile_application,False
3,5,172896,2019,8,18.111082,102.629832,Lao People's Democratic Republic,1,8,4,1,2,ground,NaN,mobile_application,False
4,7,172850,2013,1,14.462585,99.477376,Thailand,1,0,0,1,0,streetview,NaN,NaN,False


In [43]:
SEED = 7
def getSample(dt, key, val, num = 20):
  return dt[dt[key] == val].sample(n = num, random_state = SEED)

Filter out rows by years we are interested in. For starters, we are only looking at data beyond 2017

In [ ]:
year = 2016
country = 'United States of America'
ydata = data[(data['year'] >= year) ] #& (data['country'] != country)
print('Number of rows: ', ydata.shape[0])
print('\nLabel Distribution')
print('-'*60)
print('Number of rows with irrigated land: ', ydata[ydata['water'] == 1].shape[0])
print('Number of rows with rainfed land: ', ydata[ydata['water'] == 2].shape[0])

Number of rows:  5

Label Distribution
------------------------------------------------------------
Number of rows with irrigated land:  4
Number of rows with rainfed land:  1


In [ ]:
# Pick 20 random rows from irrigated lands and pick 20 random from non irrigated
irr_sample = getSample(dt = ydata, key = 'water', val = 1, num = 4)
print('Number of rows with irrigated land: ', irr_sample.shape[0])
irr_sample.head()

Number of rows with irrigated land:  4


,Unnamed: 0,id,year,month,lat,lon,country,land_use_type,crop_primary,crop_secondary,water,intensity,source_type,source_class,source_description,use_validation
3,5,172896,2019,8,18.111082,102.629832,Lao People's Democratic Republic,1,8,4,1,2,ground,NaN,mobile_application,False
2,4,172953,2019,8,35.177561,-111.680403,United States of America,1,0,0,1,0,ground,NaN,mobile_application,False
0,0,172978,2020,0,50.863663,4.359379,Belgium,1,0,1,1,2,ground,NaN,mobile_application,False
9,18,172821,2017,2,35.227342,-111.543740,United States of America,1,4,11,1,2,ground,NaN,mobile_application,False


In [10]:
# Pick 20 random rows on rainfed land
rf_sample = getSample(dt = ydata, key = 'water', val = 2, num = 1)
rf_sample.head()

NameError: ignored

In [44]:
year = 2015
country = 'United States of America'
ydata = data[(data['year'] >= year) & (data['country'] == country)]
print('Number of rows: ', ydata.shape[0])
print('\nLabel Distribution')
print('-'*60)
print('Number of rows with irrigated land: ', ydata[ydata['water'] == 1].shape[0])
print('Number of rows with rainfed land: ', ydata[ydata['water'] == 2].shape[0])

Number of rows:  27

Label Distribution
------------------------------------------------------------
Number of rows with irrigated land:  20
Number of rows with rainfed land:  7


In [65]:
# Pick 20 random rows from irrigated lands and pick 20 random from non irrigated
irr_sample = getSample(dt = ydata, key = 'water', val = 1, num = 10)
print('Number of rows with irrigated land: ', irr_sample.shape[0])
#irr_sample.head()
ydata

Number of rows with irrigated land:  10


,Unnamed: 0,id,year,month,lat,lon,country,land_use_type,crop_primary,crop_secondary,water,intensity,source_type,source_class,source_description,use_validation
1,1,172961,2019,8,35.606363,-112.097508,United States of America,1,20,0,2,1,ground,NaN,mobile_application,False
2,4,172953,2019,8,35.177561,-111.680403,United States of America,1,0,0,1,0,ground,NaN,mobile_application,False
9,18,172821,2017,2,35.227342,-111.543740,United States of America,1,4,11,1,2,ground,NaN,mobile_application,False
12,24,172776,2016,9,35.216925,-111.632640,United States of America,1,0,0,1,1,ground,NaN,mobile_application,False
24,50,172731,2016,7,35.402915,-111.757486,United States of America,1,3,0,1,1,ground,NaN,mobile_application,False
100,245,66462,2015,4,38.114833,-121.331518,United States of America,1,21,0,1,0,streetview,NaN,NaN,False
3185,14165,44750,2016,4,33.144888,-112.678306,United States of America,1,0,0,2,0,ground,NaN,mobile_application,False
3944,16960,1049,2016,0,35.537473,-119.262257,United States of America,1,21,0,2,0,derived,NaN,web_application,False
4012,17048,852,2015,10,35.215636,-111.634586,United States of America,1,22,0,1,4,ground,NaN,mobile_application,False
4013,17049,831,2015,10,35.285337,-111.539962,United States of America,1,22,0,1,4,ground,NaN,mobile_application,False


In [47]:
# Pick 20 random rows on rainfed land
rf_sample = getSample(dt = ydata, key = 'water', val = 2, num = 7)
rf_sample.head()

,Unnamed: 0,id,year,month,lat,lon,country,land_use_type,crop_primary,crop_secondary,water,intensity,source_type,source_class,source_description,use_validation
3944,16960,1049,2016,0,35.537473,-119.262257,United States of America,1,21,0,2,0,derived,NaN,web_application,False
4024,17080,447,2015,7,43.553851,-89.700558,United States of America,1,5,0,2,1,unknown,NaN,NaN,False
1,1,172961,2019,8,35.606363,-112.097508,United States of America,1,20,0,2,1,ground,NaN,mobile_application,False
4026,17083,429,2015,7,44.259762,-89.577066,United States of America,1,2,0,2,1,unknown,NaN,NaN,False
4014,17061,646,2015,7,35.740591,-102.871857,United States of America,1,0,0,2,0,unknown,NaN,NaN,False


## Helper Methods

Below are the functions we need to download an EE image for a specified co-ordinate and time

In [49]:
#export_folder = '/content/gdrive/My Drive/capstone_code/CroplandDataSet/'
class GetEEImage():

    def __init__(self, center_lat, center_lon, edge_len = 0.005, year = 2020, month = 6, water=2, byyear = True):
        '''
        Parameters:
            center_lat: latitude for the location coordinate
            center_lon: longitude for the location coordinate
            edge_len: edge length in degrees for the rectangle given the location coordinates
            year: year the satellite data should pull images for
            '''

        # Initialize the library.
        ee.Initialize()

        # Error handle parameter issues
        if center_lat >= -90 and center_lat <= 90:
            self.center_lat = center_lat
        else:
            raise ValueError('Please enter float value for latitude between -90 and 90')
            exit()

        if center_lon >= -180 and center_lon <= 180:
            self.center_lon = center_lon
        else:
            raise ValueError('Please enter float value for longitude between -180 and 180')
            exit()

        if (type(edge_len) == float and (edge_len <= 0.5 and edge_len >= 0.005)):
            self.edge_len = edge_len
        else:
            raise ValueError('Please enter float value for edge length between 0.5 and 0.005')
            exit()

        # (range is 2017 to year prior)
        if ((type(year) == int) and (year >= 2015 and year <= int(time.strftime("%Y")) - 1)):
            self.year = year
        else:
            raise ValueError(
                'Please enter an integer value for year > 2017 and less than the current year')
            exit()


        #month
        self.month = month
        self.byyear = byyear

        # (range is 1 to 2)
        if (type(water) == int) and not (water == 1 or water == 2):
            raise ValueError(
                'Please enter a valid integer value for water (1,2)')
            exit()

        # initialize remaining variables
        self.label = []
        self.comment = dict()
        self.image = ee.Image()
        self.simple_image = ee.Image()
        self.base_asset_directory = None

        # Create the bounding box using GEE API
        self.aoi_ee = self.__create_bounding_box_ee()
        # Estimate the area of interest
        self.dist_lon = self.__calc_distance(
            self.center_lon - self.edge_len / 2, self.center_lat, self.center_lon + self.edge_len / 2, self.center_lat)
        self.dist_lat = self.__calc_distance(
            self.center_lon, self.center_lat - self.edge_len / 2, self.center_lon, self.center_lat + self.edge_len / 2)
        print('The selected area is approximately {:.2f} km by {:.2f} km'.format(
            self.dist_lon, self.dist_lat))

        self.model_projection = "EPSG:3857" #Mercator projection for the map image

    def __create_bounding_box_ee(self):
        '''Creates a rectangle for pulling image information using center coordinates and edge_len'''
        return ee.Geometry.Rectangle([self.center_lon - self.edge_len / 2, self.center_lat - self.edge_len / 2, self.center_lon + self.edge_len / 2, self.center_lat + self.edge_len / 2])

    def __create_bounding_box_shapely(self):
        '''Returns a box for coordinates to plug in as an image add-on layer'''
        return box(self.center_lon - self.edge_len / 2, self.center_lat - self.edge_len / 2, self.center_lon + self.edge_len / 2, self.center_lat + self.edge_len / 2)

    @staticmethod
    def __calc_distance(lon1, lat1, lon2, lat2):
        '''Calculates the distance between 2 coordinates'''
        # Reference: https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude
        # approximate radius of earth in km
        R = 6373.0
        lon1 = radians(lon1)
        lat1 = radians(lat1)
        lon2 = radians(lon2)
        lat2 = radians(lat2)
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        distance = R * c
        return distance

    def __maskL8sr(self, image):
      '''
      This function returns a mask for LANDSAT8 surface reflectance by looking for
      clear conditions for cloud shadow and cloud
      '''
      # Bits 3 and 5 are cloud shadow and cloud, respectively.
      cloudShadowBitMask = (1 << 3);
      cloudsBitMask = (1 << 5);
      # Get the pixel QA band.
      qa = image.select('pixel_qa');
      # Both flags should be set to zero, indicating clear conditions.
      mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                    .And(qa.bitwiseAnd(cloudsBitMask).eq(0));
      return image.updateMask(mask);
    

    def pull_Sentinel2_data(self):
       
        
        # 10 of 13 Spectral Bands are retained. 10th band has no surface reflectance per
        # http://bigearth.net/static/documents/BigEarthNet_IGARSS_2019.pdf
        
        # Also the baseline model only used the 10 and 20m bands (remove band 1 and 9)
        band_names = ['B2', 'B3', 'B4', 'B5',
                      'B6', 'B7']
 
        if not self.byyear:
          if self.month and self.month > 0 and self.month < 12:
            random_month = self.month
          else:
            random_month = np.random.randint(1,13)

          start_date = f'{self.year}-{random_month}-01'
          if random_month != 12:
              end_date = f'{self.year}-{random_month+1}-01'
          else:
              end_date = f'{self.year +1 }-1-01'
        else:
          random_month = 0
          start_date = f'{self.year}-01-01'
          end_date = f'{self.year}-12-31'

        self.Sentinel_MSI = (ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
                             .filterDate(start_date, end_date)
                             .filterBounds(self.aoi_ee)
                             .map(self.__maskL8sr)
                             .select(band_names)
                             .median().clip(self.aoi_ee))
        return random_month


    def plot_map(self):
        '''Plot folium map using GEE api - the map includes are of interest box and associated ndvi readings'''

        def add_ee_layer(self, ee_object, vis_params, show, name):
            '''Checks if image object classifies as ImageCollection, FeatureCollection, Geometry or single Image
            and adds to folium map accordingly'''
            try:
                if isinstance(ee_object, ee.image.Image):
                    map_id_dict = ee.Image(ee_object).getMapId(vis_params)
                    folium.raster_layers.TileLayer(
                        tiles=map_id_dict['tile_fetcher'].url_format,
                        attr='Google Earth Engine',
                        name=name,
                        overlay=True,
                        control=True,
                        show=show
                    ).add_to(self)
                elif isinstance(ee_object, ee.imagecollection.ImageCollection):
                    ee_object_new = ee_object.median()
                    map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
                    folium.raster_layers.TileLayer(
                        tiles=map_id_dict['tile_fetcher'].url_format,
                        attr='Google Earth Engine',
                        name=name,
                        overlay=True,
                        control=True,
                        show=show
                    ).add_to(self)
                elif isinstance(ee_object, ee.geometry.Geometry):
                    folium.GeoJson(
                        data=ee_object.getInfo(),
                        name=name,
                        overlay=True,
                        control=True
                    ).add_to(self)
                elif isinstance(ee_object, ee.featurecollection.FeatureCollection):
                    ee_object_new = ee.Image().paint(ee_object, 0, 2)
                    map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
                    folium.raster_layers.TileLayer(
                        tiles=map_id_dict['tile_fetcher'].url_format,
                        attr='Google Earth Engine',
                        name=name,
                        overlay=True,
                        control=True,
                        show=show
                    ).add_to(self)

            except:
                print("Could not display {}".format(name))

        # Add EE drawing method to folium.
        folium.Map.add_ee_layer = add_ee_layer

        myMap = folium.Map(location=[self.center_lat, self.center_lon], zoom_start=11)
        #aoi_shapely = self.__create_bounding_box_shapely()
        #folium.GeoJson(aoi_shapely, name="Area of Interest").add_to(myMap)

        
        # Add Sentinel-2 RGB quarterly layers
        start = time.time()
        visParams = {'max': 3000}
        # Add MSI layer for July
        myMap.add_ee_layer(self.Sentinel_MSI.select(['B2','B3','B4']), visParams, show=False, name="Sentinel2A")
        end = time.time()
        print("ADDED S2 RGB LAYERS \t\t--> " + str(round((end - start) / 60, 2)) + " min")

        return myMap


    def write_image_google_drive(self, filename, dirname):
        '''Writes predicted image out as an image to Google Drive as individual TIF files
        They will need to be combined after the fact'''
        bands = ['B2', 'B3', 'B4', 'B5',
                  'B6', 'B7']
        tasks = []
                
        for band in bands:
            tasks.append(ee.batch.Export.image.toDrive(
                folder = dirname,
                crs=self.model_projection,
                region=self.aoi_ee,
                image=self.Sentinel_MSI.select(band),
                description=f'{filename}_msi_{band}',
                scale = 10,
                maxPixels=1e13))
        
        print(f"Writing To Google Drive in folder = {filename}")
        
        for t in tasks:
                t.start()

## Degree to Distance Calculation

Source: https://www.usgs.gov/faqs/how-much-distance-does-a-degree-minute-and-second-cover-your-maps?qt-news_science_products=0#qt-news_science_products

- One degree of latitude equals approximately 364,080 feet (69 miles), one minute equals 6,068 feet (1.15 miles), and one-second equals 101 feet.
- One-degree of longitude equals 288,200 feet (54.6 miles), one minute equals 4,800 feet (0.91 mile), and one second equals 80 feet.
- 1.60934 km per mile
- 9748 square kilometers per squared degree


## Download Images from EE to Drive

The code below will download the images from EE to the specified folder in Google drive. You dont need to specify the path to the folder as long as the folder name is unique and found in the working directory that you have mentioned above.

In [61]:
from os import mkdir

# Latitude and Longitude of center point
edge_len = 0.25

def downloadEarthData(dt):
  
  # Iterate over our entries in the ydata frame and download the specific image
  for idx, row in dt.iterrows():

    print(f'Idx: {idx}')
    if idx == 3944:
      continue
    
    # Instantiate the model
    lat = row['lat']
    lon = row['lon']
    year = row['year']
    month = row['month']
    water = row['water']

    print(f'Evaluating irrigation at index {idx} : [{lat}, {lon}] for month {month} in year {year} with label: {water}')
    
    # Instantiate the model
    model = GetEEImage(
        center_lat = lat, 
        center_lon = lon, 
        edge_len = edge_len, 
        year = year,
        month = month,
        water = water,
        byyear = True)
    
    month = model.pull_Sentinel2_data()
              
    base_filename = f'S2SR_{month}_{year}_{lat}_{lon}' 

    dirname = 'USIrrigated' if water == 1 else 'USRainfed'
    #rgb_img = model.Sentinel_MSI.select(['B2', 'B3', 'B4'])
    #rgb_img.getInfo()


    model.write_image_google_drive(base_filename, dirname)


In [64]:
# Download data for nonUS irr data
downloadEarthData(irr_sample)

Idx: 9
Evaluating irrigation at index 9 : [35.22734228, -111.5437402] for month 2 in year 2017 with label: 1
The selected area is approximately 22.72 km by 27.81 km
Writing To Google Drive in folder = S2SR_0_2017_35.22734228_-111.5437402
Idx: 4029
Evaluating irrigation at index 4029 : [35.21564979999999, -111.63351862] for month 4 in year 2015 with label: 1
The selected area is approximately 22.72 km by 27.81 km
Writing To Google Drive in folder = S2SR_0_2015_35.21564979999999_-111.63351862
Idx: 12
Evaluating irrigation at index 12 : [35.21692454, -111.63264005] for month 9 in year 2016 with label: 1
The selected area is approximately 22.72 km by 27.81 km
Writing To Google Drive in folder = S2SR_0_2016_35.21692454_-111.63264005
Idx: 4012
Evaluating irrigation at index 4012 : [35.21563560000001, -111.63458572] for month 10 in year 2015 with label: 1
The selected area is approximately 22.72 km by 27.81 km
Writing To Google Drive in folder = S2SR_0_2015_35.21563560000001_-111.63458572
Idx